In [1]:
import numpy as np
from collections import Counter
from conllu import parse_incr
import time
import enum
from collections import Counter
from conllu import parse_incr
from io import open
from IPython.display import display
import pandas as pd

In [2]:
def compute_transition_matrix(possible_tags, train):
    transition_matrix = np.zeros((len(possible_tags), len(possible_tags)), dtype='float32')
    
    transition_counter_dict = dict()
    counter_dict = dict()
    count_initial_dict = dict()
    count_final_dict = dict()
    
    for tag1 in possible_tags:
        counter_dict[tag1] = 0
        count_initial_dict[tag1] = 0
        count_final_dict[tag1] = 0
        for tag2 in possible_tags:
            transition_counter_dict[(tag1, tag2)] = 0
    
    sentence_n = 0
    for sentence in parse_incr(train):
        sentence_n += 1
        for i in range(len(sentence)):
            word_before = sentence[i-1]
            word = sentence[i]
            if i == 0:
                if word["upos"] in count_initial_dict.keys():
                    count_initial_dict[word["upos"]] = count_initial_dict[word["upos"]] + 1
            if (word_before["upos"], word["upos"]) in transition_counter_dict.keys() and i != 0:
                transition_counter_dict[(word_before["upos"], word["upos"])] = transition_counter_dict[(word_before["upos"], word["upos"])] + 1
            if word["upos"] in counter_dict.keys():
                counter_dict[word["upos"]] = counter_dict[word["upos"]] + 1
            if i == len(sentence) - 1:
                if (word["upos"], 'END') in transition_counter_dict.keys():
                    transition_counter_dict[(word["upos"], 'END')] = transition_counter_dict[(word_before["upos"], word["upos"])] + 1
    
    #probabilità di transizione iniziali           
    for i,t in enumerate(possible_tags):
        transition_matrix[0][i] = count_initial_dict[t]/sentence_n
    #probabilità di transizione intermedie
    for i,t1 in enumerate(possible_tags):
        for j,t2 in enumerate(possible_tags):
            if i >= 1 and j >= 1 and i < (len(possible_tags) - 1):
                transition_matrix[i][j] =  transition_counter_dict[(t1,t2)]/counter_dict[t1]
    #probabilità di transizione finali
    """for i,t in enumerate(possible_tags):
        transition_matrix[i][len(possible_tags) - 1] = count_final_dict[t]/sentence_n"""
    
    train.seek(0)
    return transition_matrix

In [3]:
class Smoothing(enum.Enum):
    UNKNOWN_NAME = 1
    UNKNOWN_NAME_VERB = 2
    UNKNOWN_ALL = 3
    UNKNOWN_DISTRIBUTION_ONESHOT_WORDS = 4
    
class Language(enum.Enum):
    GREEK = 1
    LATIN = 2 

In [4]:
start = ['START']

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

smoothing_strategy = Smoothing.UNKNOWN_NAME_VERB
language = Language.LATIN

if language.name == 'GREEK':
    train = open("grc_perseus-ud-train.conllu", "r", encoding="utf-8")
    dev = open("grc_perseus-ud-dev.conllu","r", encoding="utf-8")
    test = open("grc_perseus-ud-test.conllu", "r", encoding="utf-8")
    possible_tags = ['START','ADJ','ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN',
                     'NUM', 'PART', 'PRON','SCONJ', 'VERB', 'X', 'PUNCT','END']
elif language.name == 'LATIN':
    train = open("la_llct-ud-train.conllu", "r", encoding="utf-8")
    dev = open("la_llct-ud-dev.conllu", "r", encoding="utf-8")
    test = open("la_llct-ud-test.conllu", "r", encoding="utf-8")
    possible_tags = ['START','ADJ','ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'NOUN',
                     'NUM', 'PART', 'PRON','PROPN','PUNCT', 'SCONJ', 'VERB','X','END']

In [5]:
transition_matrix = pd.DataFrame(compute_transition_matrix(possible_tags, train), columns = list(possible_tags), index=list(possible_tags))

In [6]:
display(transition_matrix)

,START,ADJ,ADP,ADV,AUX,CCONJ,DET,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,VERB,X,END
START,0.0,0.046508,0.022911,0.070243,0.000412,0.191384,0.034984,0.139388,0.001372,0.000549,0.025792,0.009878,0.378241,0.003841,0.074359,0.000137,0.000000
ADJ,0.0,0.089186,0.022273,0.005023,0.034594,0.047578,0.006729,0.283196,0.000758,0.000284,0.008435,0.213819,0.252867,0.000569,0.034499,0.000190,0.000000
ADP,0.0,0.090429,0.002025,0.008323,0.000000,0.000000,0.244686,0.353729,0.006355,0.000000,0.105331,0.147677,0.000562,0.000169,0.040659,0.000056,0.000000
ADV,0.0,0.011566,0.214399,0.048576,0.017782,0.013879,0.043516,0.080382,0.005060,0.029059,0.061009,0.129536,0.032529,0.010843,0.301720,0.000145,0.000000
AUX,0.0,0.089726,0.109915,0.013459,0.000000,0.026021,0.018394,0.061014,0.001795,0.000449,0.154329,0.014805,0.275011,0.005384,0.229699,0.000000,0.000000
CCONJ,0.0,0.054470,0.157828,0.050239,0.004051,0.007563,0.089943,0.263798,0.029621,0.010354,0.037364,0.054560,0.004952,0.059242,0.175925,0.000090,0.000000
DET,0.0,0.061589,0.056543,0.076478,0.001415,0.024611,0.055128,0.447487,0.007629,0.000554,0.033225,0.076724,0.060604,0.014705,0.083308,0.000000,0.000000
NOUN,0.0,0.101177,0.092977,0.023564,0.007742,0.076263,0.151512,0.082509,0.018716,0.000892,0.027326,0.077879,0.180141,0.007091,0.151778,0.000434,0.000000
NUM,0.0,0.015735,0.189908,0.034183,0.003256,0.051546,0.010309,0.421595,0.022789,0.000000,0.003798,0.000543,0.185567,0.001085,0.059685,0.000000,0.000000
PART,0.0,0.000000,0.015009,0.011257,0.223265,0.043152,0.001876,0.071295,0.000000,0.000000,0.009381,0.001876,0.000000,0.000000,0.622889,0.000000,0.000000
